In [1]:
#https://shotanuki.com/python%E3%81%A7pdf%E3%82%92%E7%94%BB%E5%83%8F%E3%81%AB%E5%A4%89%E6%8F%9B%E3%81%99%E3%82%8B/
#PythonでPDFを画像に変換する
from pdf2image import convert_from_path
import datetime
import os
now = datetime.date.today()

# カレントディレクトリの全画像をループする
for pdfname in os.listdir('./mergedTRF/'):# ❶
    ext = os.path.splitext(pdfname)
    #if ext == '.pdf':
    images = convert_from_path('./mergedTRF/'+pdfname)
    os.makedirs('./trf/'+str(now), exist_ok=True)
    i = 0
    for image in images:
        image.save('./trf/'+str(now) +'/trf_' + str(pdfname)+'_'+str(i)+'.png', 'png')
        i += 1
       

In [2]:
checkmark = [
    (1185,1235,90,140),#US 1
    (1185,1235,260,310),#LMP 2
    (1185,1235,455,505), #IVF 
    (1185,1235,645,695),#PE 
    (1280,1330,65,115), #自然妊娠 
    (1330,1380,65,115), #新鮮胚利用
    (1380,1430,60,110), #融解胚利用
     (1425,1475,60,110), #提供卵子利用
    # 4 胎児数
    (1725,1775,80,130), #単胎 10
    (1730,1780,215,265), #双胎
    # 5 人種
     (1860,1910,55,105), # Japanese
     (1905,1955,55,105), # Caucasian
    (1945,1995,55,105),# African American
    (1990,2040,55,105), # Other
    # IDDM
 (995,1045,940,990), # OIDDM No
 (995,1045,1120,1170), # OIDDM Yes
    #T21
 (1145,1195,940,990), # DS No
(1145,1195,1120,1170), # DS Yes
    #NTD
 (1585,1635,935,985), # DS No 20
 (1665,1715,935,985), # DS yes
    #NTD relation
 (1785,1835,1070,1120), # Patient herself
 (1835,1885,1070,1120), # Husbond of Patient 
(1880,1930,1070,1120), # Mother of
 (1925,1975,1070,1120), # siblings 25
 (1970,2020,1070,1120), # previous child 26
    ]

old_checkmark = [
    (1155,1205,175,225),#US 1
    (1155,1205,325,375),#LMP 2
    (1155,1205,510,560), #IVF 
    (1155,1205,700,750),#PE 
    # 4 胎児数
(1725,1775,80,130), #単胎 10
(1730,1780,215,265), #双胎
    # 5 人種
 (1895,1945,55,105), # Japanese
 (1895,1945,300,350), # Caucasian
(1895,1945,565,615),# African American
(1980,2030,55,105), # Other
    # IDDM
 (995,1045,940,990), # OIDDM No
 (995,1045,1120,1170), # OIDDM Yes
    #T21
 (1135,1185,940,990), # DS No
(1135,1185,1120,1170), # DS Yes
    #NTD
 (1315,1365,935,985), # NTD No 20
 (1315,1365,1135,1185), # NTD yes
    #NTD relation
     (1485,1535,1125,1175), # NTD
 (1700,1750,1125,1175), # Patient herself
 (1700,1750,1325,1375), # Husbond of Patient 
    (1835,1885,1125,1175), # Mother of Patient 
(1840,1890,1125,1175), # Mother of hus
    (1920,1970,1125,1175), #sib of hus
 (1920,1970,1325,1375), # siblings 25
 (1975,2025,1125,1175), # previous child 26
    ]

In [3]:
#PythonでOCRがんばる(1) hhttp://sakataharumi.hatenablog.jp/entry/2016/09/16/20160916040943/
#tesseractでOCR〜数字だけでも〜 http://sakataharumi.hatenablog.jp/entry/2016/09/16/20160916052946/
# p87 17.3.2 ステップ2：全ファイルをループして画像を開く https://www.oreilly.co.jp/pub/9784873117782/auto-sample.pdf

from PIL import Image
import cv2
import sys,os
import pyocr
import pyocr.builders
import csv
import datetime
now = datetime.date.today()
now_time = datetime.datetime.today()
now_time = str(now_time.hour)+'-'+str(now_time.minute)+'-'+str(now_time.second)

%matplotlib inline
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt

os.makedirs('./results/checkmark_results/'+ str(now), exist_ok = True)
os.makedirs('./results/checkmark_results/' + str(now) + '/Error', exist_ok = True)
os.makedirs('./results/checkmark_results/' + str(now) + '/N', exist_ok = True)
os.makedirs('./results/checkmark_results/' + str(now) + '/Y', exist_ok = True)
os.makedirs('./csv/'+ str(now), exist_ok = True)
os.makedirs('./results/PtID_results/'+ str(now), exist_ok = True)

# カレントディレクトリの全画像をループする
for filename in os.listdir('./trf/'+ str(now)+'/'):# ❶
    ext = os.path.splitext(filename)
    if ext == '.png' or u'.jpeg' or u'.jpg':
        img = Image.open('./trf/'+ str(now)+'/' + filename)  # ❹
        img = img.crop((800,129,1050,200))
        img.save('./results/PtID_results/' + str(now)+'/' + "PtID_" + str(filename))

        tools = pyocr.get_available_tools()
        if len(tools) == 0:
            print("No OCR tool found")
            sys.exit(1)
        # The tools are returned in the recommended order of usage
        # pyocr http://motojapan.hateblo.jp/entry/2018/03/12/094636
        tool = tools[0] 
        txt = tool.image_to_string(
            Image.open('./results/PtID_results/'+ str(now)+'/' +  "PtID_" + str(filename)),
            lang="eng",
            builder=pyocr.tesseract.DigitBuilder(tesseract_layout=6),
        )
            # 6 = Assume a single uniform block of text.            
        #img.save('./results/PtID_results/' + str(now)+'/' + "PtID_" + str(now_time)+'_'+ str(filename))
        path_w = './csv/'+ str(now)+'/'+str(now)+'_'+str(now_time)+'.csv'
        with open(path_w, mode='a') as f:
            f.write(filename+'\t'+'checkmark\t')
            f.write(txt+'\t')
            
        img = cv2.imread('./trf/'+str(now)+'/' + filename)  # ❹
        
        # モデルを読込 --- (*1)
        labels1 = ["Error ", "N ", "Y "]
        model1 = load_model('CNN-mk-model.h5')
        model1.load_weights('CNN-mk-weight.h5')  
        
        img = img[1785:1835,1070:1120]
        res = model1.predict(np.array([img]), batch_size=32)
        idx1 = res.argmax()
            
        if (idx1 == 0):
            for j, pt in enumerate(old_checkmark):
                x,y,w,h = pt
                img = cv2.imread('./trf/'+str(now)+'/' + filename)  # ❹
                img = img[x:y,w:h]
                res = model1.predict(np.array([img]), batch_size=32)
                idx1 = res.argmax()
                cv2.imwrite('./results/checkmark_results/' + str(now) + '/'+ \
                                    str(labels1[idx1].strip())+\
                                    '/' + str(now_time)+ 'mk_'+ str(j)+'_'+ filename, img)
                j += 1
                with open(path_w, mode='a') as f:
                    f.write(str(labels1[idx1])+'\t')
        else:
            for k, pt in enumerate(checkmark):
                x,y,w,h = pt
                img = cv2.imread('./trf/'+str(now)+'/' + filename)  # ❹
                img = img[x:y,w:h]
                res = model1.predict(np.array([img]), batch_size=32)
                idx1 = res.argmax()
                cv2.imwrite('./results/checkmark_results/' + str(now) + '/'+\
                                str(labels1[idx1].strip())+\
                                '/' + str(now_time)+ 'mk_'+ str(k)+'_'+ filename, img)
                k += 1
                with open(path_w, mode='a') as f:
                    f.write(str(labels1[idx1])+'\t')
        with open(path_w, mode='a') as f:
            f.write('\n')
import shutil
shutil.move('./results/PtID_results/' + str(now)+'/',\
            './results/PtID_results/'+str(now)+'_'+ str(now_time)+'/')
shutil.move('./results/checkmark_results/' + str(now) + '/',\
            './results/checkmark_results/' + str(now) + '_'+str(now_time)+'/')
#shutil.move('./trf/'+str(now)+'/','./finished_trf/checkmark/'+str(now)+'_'+str(now_time)+'/')

Using TensorFlow backend.


'./results/checkmark_results/2019-09-04_12-1-35/'

In [4]:
 
HspID = [(620,750,30,500)] #HspID
DOB_MM = [(640,750,1025,1145)]#DOB MM
DOB_DD = [(640,750,1155,1275)]#DOB DD
DOB_YY = [(640,750,1320,1440)]#DOB YY
DD_MM = [(860,940,440,530)] #DD MM
DD_DD = [(860,940,555,645)] #DD DD
DD_YY = [(860,940,725,815)] #DD YY
EDC_MM_Old = [(1055,1165,285,375)] #EDC MM Old   
EDC_MM = [ (1055,1165,185,275)] #EDC MM New
EDC_DD_Old = [(1055,1155,415,525)] #EDC DD Old
EDC_DD = [(1055,1155,295,405)] #EDC DD New
EDC_YY_Old = [(1055,1155,585,675)] #EDC DD Old
EDC_YY = [(1055,1155,475,565)] #EDC YY  New
Donor_YY_Old = [(1310,1410,590,690)] #Donor YY Old
Donor_YY =[ (1360,1450,550,650)] #Donor YY New
Donor_MM_Old = [(1320,1410,770,870)] #Donor MM Old
Donor_MM = [(1370,1450,690,790)] #Donor MM New
Weight = [(1530,1645,415,560)] #Weight

w1 = 20
w2 = 50
h1 = 60
h2 = 80

%matplotlib inline
from keras.models import load_model
import numpy as np
import cv2
import csv
import matplotlib.pyplot as plt
from PIL import Image
import sys,os
import datetime
now = datetime.date.today()
now_time = datetime.datetime.today()
now_time = str(now_time.hour)+'-'+str(now_time.minute)+'-'+str(now_time.second)

#os.makedirs('./trf/'+str(now), exist_ok=True)
os.makedirs('./number/'+ str(now), exist_ok = True)
os.makedirs('./number/' + str(now) + '/10', exist_ok = True)
os.makedirs('./number/' + str(now) + '/0', exist_ok = True)
os.makedirs('./number/' + str(now) + '/1', exist_ok = True)
os.makedirs('./number/' + str(now) + '/2', exist_ok = True)
os.makedirs('./number/' + str(now) + '/3', exist_ok = True)
os.makedirs('./number/' + str(now) + '/4', exist_ok = True)
os.makedirs('./number/' + str(now) + '/5', exist_ok = True)
os.makedirs('./number/' + str(now) + '/6', exist_ok = True)
os.makedirs('./number/' + str(now) + '/7', exist_ok = True)
os.makedirs('./number/' + str(now) + '/8', exist_ok = True)
os.makedirs('./number/' + str(now) + '/9', exist_ok = True)
os.makedirs('./results/number_results/' + str(now), exist_ok = True)
#os.makedirs('./csv/'+ str(now), exist_ok = True)
#path_w = './csv/'+ str(now)+'/'+str(now)+'_'+str(now_time)+'.csv'

labels = ["0","1","2","3","4","5","6","7","8","9","Error"]
model = load_model('CNN-number-model.h5')
model.load_weights('CNN-number-weight.h5')

labels1 = ["Error ", "N ", "Y "]
model1 = load_model('CNN-mk-model.h5')
model1.load_weights('CNN-mk-weight.h5')    

In [5]:
# 依頼書画像から番号領域を抽出する関数
def detect_HspID(fname):
    for j, pt in enumerate(HspID):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not w1 < w < w2: continue
                if not h1 < h < h2: continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 30: continue
                result2.append([x, y, w, h])
                lastx = x 
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img 
            

In [6]:
# 依頼書画像から番号領域を抽出する関数
def detect_DOB_MM(fname):
    for j, pt in enumerate(DOB_MM):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not w1 < w < w2: continue
                if not h1 < h < h2: continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 30: continue
                result2.append([x, y, w, h])
                lastx = x 
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img 
            

In [7]:
# 依頼書画像から番号領域を抽出する関数
def detect_DOB_DD(fname):
    # 画像を読み込
    for j, pt in enumerate(DOB_DD):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not w1 < w < w2: continue
                if not h1 < h < h2: continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 30: continue
                result2.append([x, y, w, h])
                lastx = x 
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img 
     

In [8]:
 # 依頼書画像から番号領域を抽出する関数
def detect_DOB_YY(fname):
    # 画像を読み込む
    for j, pt in enumerate(DOB_YY):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not w1 < w < w2: continue
                if not h1 < h < h2: continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 30: continue
                result2.append([x, y, w, h])
                lastx = x 
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img 
     

In [9]:
 # 依頼書画像から番号領域を抽出する関数
def detect_DD_MM(fname):
    # 画像を読み込む
    for j, pt in enumerate(DD_MM):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not w1 < w < w2: continue
                if not h1 < h < h2: continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 30: continue
                result2.append([x, y, w, h])
                lastx = x 
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img 
    

In [10]:
 # 依頼書画像から番号領域を抽出する関数
def detect_DD_DD(fname):
    # 画像を読み込む
    for j, pt in enumerate(DD_DD):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not w1 < w < w2: continue
                if not h1 < h < h2: continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 30: continue
                result2.append([x, y, w, h])
                lastx = x 
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img  # 依頼書画像から番号領域を抽出する関数
def detect_DD_DD(fname):
    # 画像を読み込む
    for j, pt in enumerate(DD_DD):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not w1 < w < w2: continue
                if not h1 < h < h2: continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 30: continue
                result2.append([x, y, w, h])
                lastx = x 
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img 

In [11]:
 # 依頼書画像から番号領域を抽出する関数
def detect_DD_YY(fname):
    # 画像を読み込む
    for j, pt in enumerate(DD_YY):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not w1 < w < w2: continue
                if not h1 < h < h2: continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 30: continue
                result2.append([x, y, w, h])
                lastx = x 
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img 
       

In [12]:

# 依頼書画像から番号領域を抽出する関数
def detect_EDC_MM(fname):
    # 画像を読み込む

    img0 = cv2.imread(fname)
    img0 = img0[1785:1835,1070:1120]
    res = model1.predict(np.array([img0]), batch_size=32)
    idx1 = res.argmax()  
    
    if idx1 == 0:
        for j, pt in enumerate(EDC_MM_Old):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not w1 < w < w2: continue
                if not h1 < h < h2: continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 30: continue
                result2.append([x, y, w, h])
                lastx = x 
                    
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img
    
      
    else:
        for j, pt in enumerate(EDC_MM):
                img = cv2.imread(fname)
                x,y,w,h = pt
                img = img[x:y,w:h]
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                gray = cv2.GaussianBlur(gray, (3, 3), 0) 
                #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
                im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
                # 輪郭を抽出 --- (*3)
                cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
                # 抽出した輪郭を単純なリストに変換--- (*4)
                result = []
                for k in cnts:
                    x, y, w, h = cv2.boundingRect(k)
                    # 大きすぎる小さすぎる領域を除去 --- (*5)
                    if not w1 < w < w2: continue
                    if not h1 < h < h2: continue
                    result.append([x, y, w, h])
                # 抽出した輪郭が左側から並ぶようソート --- (*6)
                result = sorted(result, key=lambda x: x[0])
                # 抽出した輪郭が近すぎるものを除去 --- (*7)
                result2 = []
                lastx = -100
                for x, y, w, h in result:
                    if (x - lastx) < 30: continue
                    result2.append([x, y, w, h])
                    lastx = x 
                 # 緑色の枠を描画 --- (*8)
                for x, y, w, h in result2:
                    cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
                return result2, img 
            

In [13]:
# 依頼書画像から番号領域を抽出する関数
def detect_EDC_DD(fname):
    # 画像を読み込む
    
    img0 = cv2.imread(fname)
    img0 = img0[1785:1835,1070:1120]
    res = model1.predict(np.array([img0]), batch_size=32)
    idx1 = res.argmax()  
    
    if idx1 == 0:
        for j, pt in enumerate(EDC_DD_Old):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not w1 < w < w2: continue
                if not h1 < h < h2: continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 30: continue
                result2.append([x, y, w, h])
                lastx = x 
                    
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img
    
      
    else:
        for j, pt in enumerate(EDC_DD):
                img = cv2.imread(fname)
                x,y,w,h = pt
                img = img[x:y,w:h]
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                gray = cv2.GaussianBlur(gray, (3, 3), 0) 
                #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
                im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
                # 輪郭を抽出 --- (*3)
                cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
                # 抽出した輪郭を単純なリストに変換--- (*4)
                result = []
                for k in cnts:
                    x, y, w, h = cv2.boundingRect(k)
                    # 大きすぎる小さすぎる領域を除去 --- (*5)
                    if not w1 < w < w2: continue
                    if not h1 < h < h2: continue
                    result.append([x, y, w, h])
                # 抽出した輪郭が左側から並ぶようソート --- (*6)
                result = sorted(result, key=lambda x: x[0])
                # 抽出した輪郭が近すぎるものを除去 --- (*7)
                result2 = []
                lastx = -100
                for x, y, w, h in result:
                    if (x - lastx) < 30: continue
                    result2.append([x, y, w, h])
                    lastx = x 
                 # 緑色の枠を描画 --- (*8)
                for x, y, w, h in result2:
                    cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
                return result2, img 
            

In [14]:

def detect_EDC_YY(fname):
    # 画像を読み込む
    img0 = cv2.imread(fname)
    img0 = img0[1785:1835,1070:1120]
    res = model1.predict(np.array([img0]), batch_size=32)
    idx1 = res.argmax()  
    
    if idx1 == 0:
        for j, pt in enumerate(EDC_YY_Old):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not w1 < w < w2: continue
                if not h1 < h < h2: continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 30: continue
                result2.append([x, y, w, h])
                lastx = x 
                    
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img
    
      
    else:
        for j, pt in enumerate(EDC_YY):
                img = cv2.imread(fname)
                x,y,w,h = pt
                img = img[x:y,w:h]
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                gray = cv2.GaussianBlur(gray, (3, 3), 0) 
                #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
                im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
                # 輪郭を抽出 --- (*3)
                cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
                # 抽出した輪郭を単純なリストに変換--- (*4)
                result = []
                for k in cnts:
                    x, y, w, h = cv2.boundingRect(k)
                    # 大きすぎる小さすぎる領域を除去 --- (*5)
                    if not w1 < w < w2: continue
                    if not h1 < h < h2: continue
                    result.append([x, y, w, h])
                # 抽出した輪郭が左側から並ぶようソート --- (*6)
                result = sorted(result, key=lambda x: x[0])
                # 抽出した輪郭が近すぎるものを除去 --- (*7)
                result2 = []
                lastx = -100
                for x, y, w, h in result:
                    if (x - lastx) < 30: continue
                    result2.append([x, y, w, h])
                    lastx = x 
                 # 緑色の枠を描画 --- (*8)
                for x, y, w, h in result2:
                    cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
                return result2, img 
            

In [15]:

def detect_Donor_YY(fname):
    # 画像を読み込む
    img0 = cv2.imread(fname)
    img0 = img0[1785:1835,1070:1120]
    res = model1.predict(np.array([img0]), batch_size=32)
    idx1 = res.argmax()  
    
    if idx1 == 0:
        for j, pt in enumerate(Donor_YY_Old):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not w1 < w < w2: continue
                if not h1 < h < h2: continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 30: continue
                result2.append([x, y, w, h])
                lastx = x 
                    
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img
    
      
    else:
        for j, pt in enumerate(Donor_YY):
                img = cv2.imread(fname)
                x,y,w,h = pt
                img = img[x:y,w:h]
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                gray = cv2.GaussianBlur(gray, (3, 3), 0) 
                #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
                im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
                # 輪郭を抽出 --- (*3)
                cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
                # 抽出した輪郭を単純なリストに変換--- (*4)
                result = []
                for k in cnts:
                    x, y, w, h = cv2.boundingRect(k)
                    # 大きすぎる小さすぎる領域を除去 --- (*5)
                    if not w1 < w < w2: continue
                    if not h1 < h < h2: continue
                    result.append([x, y, w, h])
                # 抽出した輪郭が左側から並ぶようソート --- (*6)
                result = sorted(result, key=lambda x: x[0])
                # 抽出した輪郭が近すぎるものを除去 --- (*7)
                result2 = []
                lastx = -100
                for x, y, w, h in result:
                    if (x - lastx) < 30: continue
                    result2.append([x, y, w, h])
                    lastx = x 
                 # 緑色の枠を描画 --- (*8)
                for x, y, w, h in result2:
                    cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
                return result2, img 
            

In [16]:
def detect_Donor_MM(fname):
    img0 = cv2.imread(fname)
    img0 = img0[1785:1835,1070:1120]
    res = model1.predict(np.array([img0]), batch_size=32)
    idx1 = res.argmax()  
    
    if idx1 == 0:
        for j, pt in enumerate(Donor_MM_Old):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not w1 < w < w2: continue
                if not h1 < h < h2: continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 30: continue
                result2.append([x, y, w, h])
                lastx = x 
                    
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img
    
      
    else:
        for j, pt in enumerate(Donor_MM):
                img = cv2.imread(fname)
                x,y,w,h = pt
                img = img[x:y,w:h]
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                gray = cv2.GaussianBlur(gray, (3, 3), 0) 
                #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
                im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
                # 輪郭を抽出 --- (*3)
                cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
                # 抽出した輪郭を単純なリストに変換--- (*4)
                result = []
                for k in cnts:
                    x, y, w, h = cv2.boundingRect(k)
                    # 大きすぎる小さすぎる領域を除去 --- (*5)
                    if not w1 < w < w2: continue
                    if not h1 < h < h2: continue
                    result.append([x, y, w, h])
                # 抽出した輪郭が左側から並ぶようソート --- (*6)
                result = sorted(result, key=lambda x: x[0])
                # 抽出した輪郭が近すぎるものを除去 --- (*7)
                result2 = []
                lastx = -100
                for x, y, w, h in result:
                    if (x - lastx) < 30: continue
                    result2.append([x, y, w, h])
                    lastx = x 
                 # 緑色の枠を描画 --- (*8)
                for x, y, w, h in result2:
                    cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
                return result2, img 
            

In [17]:
def detect_Wt(fname):
    # 画像を読み込む
    img0 = cv2.imread(fname)
    img0 = img0[1785:1835,1070:1120]
    res = model1.predict(np.array([img0]), batch_size=32)
    idx1 = res.argmax()  
    
    if idx1 == 0:
        for j, pt in enumerate(Weight):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not w1 < w < w2: continue
                if not h1 < h < h2: continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 30: continue
                result2.append([x, y, w, h])
                lastx = x 
                    
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img
    
      
    else:
        for j, pt in enumerate(Weight):
                img = cv2.imread(fname)
                x,y,w,h = pt
                img = img[x:y,w:h]
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                gray = cv2.GaussianBlur(gray, (3, 3), 0) 
                #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
                im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
                # 輪郭を抽出 --- (*3)
                cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
                # 抽出した輪郭を単純なリストに変換--- (*4)
                result = []
                for k in cnts:
                    x, y, w, h = cv2.boundingRect(k)
                    # 大きすぎる小さすぎる領域を除去 --- (*5)
                    if not w1 < w < w2: continue
                    if not h1 < h < h2: continue
                    result.append([x, y, w, h])
                # 抽出した輪郭が左側から並ぶようソート --- (*6)
                result = sorted(result, key=lambda x: x[0])
                # 抽出した輪郭が近すぎるものを除去 --- (*7)
                result2 = []
                lastx = -100
                for x, y, w, h in result:
                    if (x - lastx) < 30: continue
                    result2.append([x, y, w, h])
                    lastx = x 
                 # 緑色の枠を描画 --- (*8)
                for x, y, w, h in result2:
                    cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
                return result2, img 

In [18]:
labels = ["0","1","2","3","4","5","6","7","8","9","Error"]
model = load_model('CNN-number-model.h5')
model.load_weights('CNN-number-weight.h5')

for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_HspID('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'HspID'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/HspID_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/HspID_'+ str(j)+'_'+ filename, img)
                if idx == 10: continue
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx]))
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [19]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_DOB_MM('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'DOB_MM'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/DOB_MM_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/DOB_MM_'+ str(j)+'_'+ filename, img)
                if idx == 10: continue
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx]))
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [20]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_DOB_DD('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'DOB_DD'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/DOB_DD_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/DOB_DD_'+ str(j)+'_'+ filename, img)
                if idx == 10: continue
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx]))
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')
                    

In [21]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_DOB_YY('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'DOB_YY'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/DOB_YY_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/DOB_YY_'+ str(j)+'_'+ filename, img)
                if idx == 10: continue
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx]))
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [22]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_DD_MM('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'DD_MM'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/DD_MM_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/DD_MM_'+ str(j)+'_'+ filename, img)
                if idx == 10: continue
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx]))
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')
                    

In [23]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_DD_DD('./trf/'+str(now)+'/'+ filename)
            if img is None : break
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'DD_DD'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/DD_DD_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/DD_DD_'+ str(j)+'_'+ filename, img)
                if idx == 10: continue
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx]))
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [24]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_DD_YY('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'DD_YY'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/DD_YY_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/DD_YY_'+ str(j)+'_'+ filename, img)
                if idx == 10: continue
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx]))
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [25]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_EDC_DD('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'EDC_DD'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/EDC_DD_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/EDC_DD_'+ str(j)+'_'+ filename, img)
                if idx == 10: continue
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx]))
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [26]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_EDC_MM('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'EDC_MM'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/EDC_MM_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/EDC_MM_'+ str(j)+'_'+ filename, img)
                if idx == 10: continue
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx]))
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [27]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_EDC_YY('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'EDC_YY'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/EDC_YY_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/EDC_YY_'+ str(j)+'_'+ filename, img)
                if idx == 10: continue
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx]))
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [28]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_Donor_YY('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'Donor_YY'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/Donor_YY_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/Donor_YY_'+ str(j)+'_'+ filename, img)
                if idx == 10: continue
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx]))
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [29]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_Donor_MM('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'Donor_MM'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/Donor_MM_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/Donor_MM_'+ str(j)+'_'+ filename, img)
                if idx == 10: continue
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx]))
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [30]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_Wt('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'Wt'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #`予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./number/'+str(now)+'/'+str(idx)+'/Wt_'+ str(j)+'_'+str(idx)+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/number_results/'+str(now)+'/Wt_'+ str(j)+'_'+ filename, img)
                if idx == 10: continue
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx]))
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [31]:
import shutil
shutil.move('./number/' + str(now)+'/','./number/'+str(now)+'_'+ str(now_time)+'/')
shutil.move('./results/number_results/' + str(now) + '/','./results/number_results/' + str(now) + '_'+str(now_time)+'/')
#shutil.move('./trf/'+str(now)+'/','./finished_trf/number/'+str(now)+'_'+str(now_time)+'/')

'./results/number_results/2019-09-04_12-12-52/'

In [32]:

Pt_Last = [(610,710,520,1030)] #Pt Last
Pt_First = [(670,770,520,1030)] #Pt First
Dr_Last = [(470,570,1095,1600)] #Dr Last
Dr_First = [(530,630,1095,1600)] #Dr First

w3 = 25
w4 = 50
h3 = 50
h4 = 80

%matplotlib inline
from keras.models import load_model
import numpy as np
import cv2
import csv
import matplotlib.pyplot as plt
from PIL import Image
import sys,os
import datetime
now = datetime.date.today()
now_time = datetime.datetime.today()
now_time = str(now_time.hour)+'-'+str(now_time.minute)+'-'+str(now_time.second)

#os.makedirs('./trf/'+str(now), exist_ok=True)

os.makedirs('./english/'+ str(now), exist_ok = True)
os.makedirs('./english/' + str(now) + '/A', exist_ok = True)
os.makedirs('./english/' + str(now) + '/B', exist_ok = True)
os.makedirs('./english/' + str(now) + '/C', exist_ok = True)
os.makedirs('./english/' + str(now) + '/D', exist_ok = True)
os.makedirs('./english/' + str(now) + '/E', exist_ok = True)
os.makedirs('./english/' + str(now) + '/F', exist_ok = True)
os.makedirs('./english/' + str(now) + '/G', exist_ok = True)
os.makedirs('./english/' + str(now) + '/H', exist_ok = True)
os.makedirs('./english/' + str(now) + '/I', exist_ok = True)
os.makedirs('./english/' + str(now) + '/J', exist_ok = True)
os.makedirs('./english/' + str(now) + '/K', exist_ok = True)
os.makedirs('./english/' + str(now) + '/L', exist_ok = True)
os.makedirs('./english/' + str(now) + '/M', exist_ok = True)
os.makedirs('./english/' + str(now) + '/N', exist_ok = True)
os.makedirs('./english/' + str(now) + '/O', exist_ok = True)
os.makedirs('./english/' + str(now) + '/P', exist_ok = True)
os.makedirs('./english/' + str(now) + '/Q', exist_ok = True)
os.makedirs('./english/' + str(now) + '/R', exist_ok = True)
os.makedirs('./english/' + str(now) + '/S', exist_ok = True)
os.makedirs('./english/' + str(now) + '/T', exist_ok = True)
os.makedirs('./english/' + str(now) + '/U', exist_ok = True)
os.makedirs('./english/' + str(now) + '/V', exist_ok = True)
os.makedirs('./english/' + str(now) + '/W', exist_ok = True)
os.makedirs('./english/' + str(now) + '/X', exist_ok = True)
os.makedirs('./english/' + str(now) + '/Y', exist_ok = True)
os.makedirs('./english/' + str(now) + '/Z', exist_ok = True)
os.makedirs('./english/' + str(now) + '/Error', exist_ok = True)
os.makedirs('./results/english_results/' + str(now), exist_ok = True)
#os.makedirs('./csv/'+ str(now), exist_ok = True)
#path_w = './csv/'+ str(now)+'/english_'+str(now)+'_'+str(now_time)+'.csv'

labels = ["A", "B", "C", "D", "E","Error", "F", "G", "H", \
          "I", "J", "K", "L", "M", "N", "O", "P", \
          "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]

model = load_model('CNN-eng-trf-model.h5')
model.load_weights('CNN-eng-trf-weight.h5')


In [33]:
# 依頼書画像から番号領域を抽出する関数
def detect_PtLastName(fname):
    # 画像を読み込む

    for j, pt in enumerate(Pt_Last):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not(w3 < w < w4): continue
                elif not(h3 < h < h4): continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 10: continue
                result2.append([x, y, w, h])
                lastx = x 
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img 
            

In [34]:
# 依頼書画像から番号領域を抽出する関数
def detect_PtFirstName(fname):
    # 画像を読み込む
    for j, pt in enumerate(Pt_First):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not(w3 < w < w4): continue
                elif not(h3 < h < h4): continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 10: continue
                result2.append([x, y, w, h])
                lastx = x 
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img 
            

In [35]:
 # 依頼書画像から番号領域を抽出する関数
def detect_DrLastName(fname):
    # 画像を読み込む
    for j, pt in enumerate(Dr_Last):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not(w3 < w < w4): continue
                elif not(h3 < h < h4): continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 10: continue
                result2.append([x, y, w, h])
                lastx = x 
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img 

In [36]:
 # 依頼書画像から番号領域を抽出する関数
def detect_DrFirstName(fname):
    # 画像を読み込む
    for j, pt in enumerate(Dr_First):
            img = cv2.imread(fname)
            x,y,w,h = pt
            img = img[x:y,w:h]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (3, 3), 0) 
            #im2 = cv2.threshold(gray, 141, 255, cv2.THRESH_BINARY_INV)[1]
            im2 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)[1]
    
            # 輪郭を抽出 --- (*3)
            cnts = cv2.findContours(im2, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
            # 抽出した輪郭を単純なリストに変換--- (*4)
            result = []
            for k in cnts:
                x, y, w, h = cv2.boundingRect(k)
                # 大きすぎる小さすぎる領域を除去 --- (*5)
                if not(w3 < w < w4): continue
                elif not(h3 < h < h4): continue
                result.append([x, y, w, h])
            # 抽出した輪郭が左側から並ぶようソート --- (*6)
            result = sorted(result, key=lambda x: x[0])
            # 抽出した輪郭が近すぎるものを除去 --- (*7)
            result2 = []
            lastx = -100
            for x, y, w, h in result:
                if (x - lastx) < 10: continue
                result2.append([x, y, w, h])
                lastx = x 
            # 緑色の枠を描画 --- (*8)
            for x, y, w, h in result2:
                cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0,255, 0), 2)
            return result2, img 
     

In [37]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_PtLastName('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'PtLastName'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))

                #予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./english/'+str(now)+'/'+str(labels[idx])+'/PtLastName_'+ str(j)+'_'+ str(labels[idx])+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/english_results/'+str(now)+'/PtLastName_'+ str(j)+'_'+ filename, img)
                if idx == 5: continue
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx]))
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [38]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_PtFirstName('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'PtFirstName'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                #予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./english/'+str(now)+'/'+str(labels[idx])+'/PtFirstName_'+ str(j)+'_'+ str(labels[idx])+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/english_results/'+str(now)+'/PtFirstName_'+ str(j)+'_'+ filename, img)
                if idx == 5: continue
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx]))
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [39]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_DrLastName('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'DrLastName'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                
                #予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./english/'+str(now)+'/'+str(labels[idx])+'/DrLastName_'+ str(j)+'_'+ str(labels[idx])+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/english_results/'+str(now)+'/DrLastName_'+ str(j)+'_'+ filename, img)
                if idx == 5: continue
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx]))
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [40]:
for filename in os.listdir('./trf/'+str(now)+'/'):# ❶
        ext = os.path.splitext(filename)
        if ext == '.png' or u'.jpeg' or u'.jpg':
            cnts, img = detect_DrFirstName('./trf/'+str(now)+'/' + filename)
            with open(path_w, mode='a') as f:
                f.write(str(filename)+'\t'+'DrFirstName'+'\t')
            for j,pt in enumerate(cnts):
                x, y, w, h = pt
                img2 = img[y:y+h,x:x+w]
                img2 = cv2.resize(img2,(50,50))
                
                #予測する
                res = model.predict(np.array([img2]), batch_size=32)
                idx = res.argmax()
                cv2.imwrite('./english/'+str(now)+'/'+str(labels[idx])+'/DrFirstName_'+ str(j)+'_'+ str(labels[idx])+'_'+str(now_time)+'_'+ filename, img2)
                cv2.imwrite('./results/english_results/'+str(now)+'/DrFirstName_'+ str(j)+'_'+ filename, img)
                if idx == 5: continue
                else:
                    with open(path_w, mode='a') as f:
                        f.write(str(labels[idx]))
                j += 1
            with open(path_w, mode='a') as f:
                    f.write('\n')

In [41]:
import shutil
shutil.move('./english/' + str(now)+'/','./english/'+str(now)+'_'+ str(now_time)+'/')
shutil.move('./results/english_results/' + str(now) + '/','./results/english_results/' + str(now) + '_'+str(now_time)+'/')
shutil.move('./trf/'+str(now)+'/','./finished_trf/'+str(now)+'_'+str(now_time)+'/')

'./finished_trf/2019-09-04_12-15-12/'